<a href="https://colab.research.google.com/github/JohnZ03/Open-L2O/blob/SpecialCaseK%3D1/trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# Set environments
%tensorflow_version 2.6

# Check python versiom
!python3 --version

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.6`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.
Python 3.7.11


In [10]:
# Import packages
import os
import tensorflow as tf
import cProfile

import matplotlib.pyplot as plt
import numpy as np

from scipy.linalg import null_space # For calculating null space of H

from scipy.optimize import fsolve, root, minimize # non-linear funtion solver

# Enable Eager Execution
# tf.executing_eagerly()

# Uncomment to see where your variables get placed
# tf.debugging.set_log_device_placement(True)

In [11]:
# Parameters
problem_def = {
  "user_count" : 20,
  "antenna_count" : 50,
}

n_user = problem_def["user_count"]
n_antenna = problem_def["antenna_count"]

In [12]:
# !!! This section still has error
# # Generate dataset of H
# H_real_dataset = tf.data.Dataset.from_tensor_slices(tf.random.normal([sample_size, n_user, n_antenna]))
# H_imag_dataset = tf.data.Dataset.from_tensor_slices(tf.random.normal([sample_size, n_user, n_antenna]))

# H_real_iter = iter(H_real_dataset)
# H_imag_iter = iter(H_imag_dataset)

# H_dataset = (H_real_dataset, H_imag_dataset)

In [13]:
def generate_H_null(problem_def, solvable_check=False):
  solvable = False
  n_user = problem_def["user_count"]
  n_antenna = problem_def["antenna_count"]

  while not solvable:
    H_real = tf.random.normal([n_user, n_antenna])
    H_imag = tf.random.normal([n_user, n_antenna])
    H = tf.complex(H_real, H_imag)
    
    H_null = null_space(np.array(H))
    H_null_real = tf.math.real(H_null)
    H_null_imag = tf.math.imag(H_null)

    if solvable_check:
      # solvable check TODO
      solvable = True
    else:
      solvable = True

  return (H_null_real, H_null_imag, H)


In [14]:
%%time
for i in range(1):
  H_i = generate_H_null(problem_def)
# print(H_i[0])

CPU times: user 4.39 ms, sys: 3.22 ms, total: 7.61 ms
Wall time: 8.2 ms


Add Gaussian Solver for testing H_null generation

In [15]:
def quadFunctionGroup(alpha, A):
  n = A.shape[2]
  y = 0
  for i in range(n):
    y += np.matmul(np.matmul(alpha, A[:,:,i]), alpha.transpose())
  return y

In [16]:
def main(problem_def):
  # Generate H with normal distribution
  user_count = problem_def["user_count"]
  antenna_count = problem_def["antenna_count"]

  # H_real = np.random.normal(size=(user_count, antenna_count))
  # H_imag = np.random.normal(size=(user_count, antenna_count))
  # H = tf.complex(H_real, H_imag)

  # H = [[1.419310150642549 - 1.147952778898594j, 0.197811053464361 + 0.722254032225002j, -0.804465956349547 - 0.666890670701386j, 0.835088165072682 - 0.082494425370955j, 0.215670086403744 - 0.438966153934773j], \
  #      [0.291584373984183 + 0.104874716016494j, 1.587699089974059 + 2.585491252616242j, 0.696624415849607 + 0.187331024578940j, -0.243715140377952 - 1.933022917850987j, -1.165843931482049 - 1.794678841455123j]]

  # H = np.array(H)

  dimNull = antenna_count - user_count
  # H_null = null_space(H)
  # H_null_real = tf.math.real(H_null)
  # H_null_imag = tf.math.imag(H_null)

  (H_null_real, H_null_imag, H) = generate_H_null(problem_def, solvable_check=False)
  
  # Both H_null and H are converted to numpy style here
  H_null = tf.complex(H_null_real, H_null_imag).numpy()
  H = H.numpy()

  H_combi1 = np.zeros((antenna_count, 2*dimNull))
  H_combi2 = np.zeros((antenna_count, 2*dimNull))
  for i in range(dimNull):
    H_combi1[:, 2*i] = H_null_real[:, i]
    H_combi1[:, 2*i+1] = -1 * H_null_imag[:, i]

    H_combi2[:, 2*i] = H_null_imag[:, i]
    H_combi2[:, 2*i+1] = H_null_real[:, i]

  A = np.zeros((2*dimNull, 2*dimNull, antenna_count))
  for i in range(antenna_count):
    A[:,:,i] = np.matmul(H_combi1[i,:].transpose(),H_combi1[i,:]) \
          + np.matmul(H_combi2[i,:].transpose(),H_combi2[i,:])

  alpha0 = np.ones((1, 2*dimNull))

  OptimizeResult = minimize(quadFunctionGroup, alpha0, args=(A))

  alphaResult = OptimizeResult.x

  # Calculate for x
  alphaRe = np.zeros(dimNull)
  alphaIm = np.zeros(dimNull)

  for i in range(dimNull):
    alphaRe[i] = alphaResult[i*2]
    alphaIm[i] = alphaResult[i*2+1]

  alphaComplex = tf.complex(alphaRe, alphaIm)

  x = H_null @ np.conjugate(alphaComplex).transpose()

  return (tf.math.real(tf.norm(tf.reduce_sum(H@x))).numpy(), x)

In [17]:
%%time
sum = 0
x_list = []
for n in range(100):
  sum += main(problem_def)[0]
  x_list.append(main(problem_def)[1])
print(sum)

6.390357732780201e-06
CPU times: user 46.1 s, sys: 18.9 s, total: 1min 4s
Wall time: 42.8 s


In [18]:
# print(alphaResult)
# print(x)
# print(H)
# print(OptimizeResult.success)
# print(OptimizeResult.fun)
# print(tf.math.real(tf.norm(tf.reduce_sum(H@x))).numpy())
print(x_list)

[array([-3.44851341e-03-0.00354028j,  4.65538341e-04+0.0019548j ,
       -8.77711425e-04-0.00447063j,  4.76298002e-03+0.00384856j,
        2.61919584e-03-0.0026099j ,  3.49361999e-03+0.00113027j,
        4.55792351e-03+0.00280606j, -9.30991662e-03-0.00327716j,
       -1.67554101e-03+0.00545052j,  1.06877666e-02+0.00167971j,
        1.99975078e-03+0.002375j  ,  3.65104380e-03-0.00660755j,
        4.62699083e-03+0.00395351j,  1.17103002e-03+0.00405128j,
       -4.64535406e-03+0.00345904j,  3.93264071e-03-0.00290968j,
        3.83980376e-03-0.00277225j,  3.12970126e-03-0.00275963j,
        2.90554903e-03-0.00111468j,  9.89654831e-04-0.00662116j,
       -2.15807791e-03-0.00389411j, -3.09804599e-03+0.00196748j,
       -4.11433656e-03+0.00197201j,  3.35721911e-05-0.00091788j,
        1.95050199e-03+0.00080986j, -2.85404192e-03-0.00153638j,
       -1.69948473e-03+0.00352754j, -9.59471772e-04+0.00138937j,
       -2.09427712e-03+0.00144818j,  2.62645297e-03+0.00106752j,
       -1.10157959e-04+0